# Creation of  synthetic spectroscopic-like time-series
# for two instruments and with two planets
## This code generates the dataset for Section 5.2 of the pyaneti II paper
## Oscar Barragán, Jul 2021

In [ ]:
#Imort modules
from __future__ import print_function, division, absolute_import
import numpy as np
#Import citlalatonac from pyaneti_extras, note that pyaneti has to be compiled in your machine
#and pyaneti has to be in your PYTHONPATH, e.g., you have to add in your bashrc file or equivalent
#export PYTHONPATH=${PYTHONPATH}:/pathtopyaneti/pyaneti
#and replacing pathtopyaneti with the location of pyaneti in your machine
from pyaneti_extras.citlalatonac import citlali
#citlalatonac is the class that creates the spectroscopic-like time-series 

### let us summon `citlalatonac` powers and create synthetic stellar data

We will use a quasi-periodic kernel (`kernel='QPK'`)
$$
    \gamma_{i,j}^{G,G} =  \exp 
    \left(
    - \frac{\sin^2\left[\pi \left(t_i - t_j \right)/P_{\rm GP}\right]}{2 \lambda_{\rm p}^2}
    - \frac{\left(t_i - t_j\right)^2}{2\lambda_{\rm e}^2}
    \right)
$$

with hyper-parameters $$\lambda_e = 20, \lambda_p = 0.3, P_{\rm GP} = 5 $$ 

given as `kernel_parameters=[20,0.5,5]`. In this case we will create 3 timeseries between 0 (`tmin`) and 60 days (`tmax`) following

$$
    S_1 =  A_1 G(t) + B_1 \dot{G}(t), \\
    S_2 =  A_2 G(t) + B_2 \dot{G}(t), \\
$$

with amplitudes $A_1 = 0.005, B_1=0.05, A_2=0.05, B_2=0.0$ (`amplitudes=[0.005,0.05,0.05,0.0]`). The last thing is to name the time-series, by default, the first one is always called `rv`, so we only need to name the second one, `time_series=['s2']`. We also say to citlalatinac that we want two time-series by passing `nseries=2`.

We can also pas a `seed` for the random number generator. Note that since we are insterested on simulating the observation if a star from two instruments, we have to pass the same `seed` number, so we will have the same GP sample for each instrument.

In [ ]:
#We will create data for a single star observed by two instruments i2 and i2
#Create instance i1
i1 = citlali(tmin=0,tmax=60,nseries=2,amplitudes=[0.005,0.05,0.05,0.0],kernel_parameters=[20,0.5,5],time_series=['s2'],seed=32)
#Create instance i2
i2 = citlali(tmin=0,tmax=60,nseries=2,amplitudes=[0.005,0.05,0.05,0.0],kernel_parameters=[20,0.5,5],time_series=['s2'],seed=32)
#Plot the time-series
i1.plot()
i2.plot()

At this point we can see that both instance i1 and i2 have generated the same two time-series, ensuring that both instruments will generate data from the same synthtetic star.

### Time to add the planetary signals

To addd a planetary signal, we have to call the `add_planet` attribute for our instances. We need to pass two elements to the attribute, the first one is a list constaining the plantary and orbital parameters

`planet_params = [T0,K,P,e,w]`, 

being `T0` the time of minimum conjunction (time of transit for transiting planets), `K` planet induced Doppler semi-amplitude, `P` is the planet orbital period, `e` is orbital eccentricty, and `w` is the angle of periastron. We also need to pass a string to label the planet 

`planet_name = planet_b`

In this example we will add two planets, the first one with parameters

$T0,K,P,e,w = 1,0.005,3,0,\pi/2$

and the second one 

$T0,K,P,e,w = 2,0.010,10,0.3,\pi/3$


In [ ]:
#let us create the lists with the both planet parameters that we will pass to the add_planet attribute
planet_1 = [1,0.005,3,0,np.pi/2]
planet_2 = [2,0.010,10,0.3,np.pi/3]

#Let us call the add_planet instance for the instrument i1 and let's call the planet 'b'
i1.add_planet(planet_1,'b')
#Let us plot the time-series to see if we have added the planetary signal
i1.plot()

The plot now shows an extra curve for the RV time-series showing the planet induced signal. The RV curve constains now the GP + 'b'

In [ ]:
#Let us now add the second planet
#Let us call the add_planet instance for the instrument i1 and let's call the planet 'c'
i1.add_planet(planet_2,'c')
#Let us plot the time-series to see if we have added the planetary signal
i1.plot()

The plot now shows an extra curve for the RV time-series showing the planet induced signals 'b' and 'c'. The RV curve constains now the GP + 'b' + 'c'

We have added the planet for the instrument i1, let us do the same for our instrument i2. We do this easily just copying the same attributes than before but for our i2 instance.

In [ ]:
#Add the planets
i2.add_planet(planet_1,'b')
i2.add_planet(planet_2,'c')
#Let's plot the time-series
i2.plot()

We note that i1 and i2 produce the same plots. This is exactly what we want, the same underlying astrophysical signal for both istruments, but the data that each istrument will provide us will have a different sampling and different white noise.

### Data for our i1 instrument

Now it is time to create data taken at the times $\mathbf{t}$ for our instrument i1. The times vector $\mathbf{t}$ can be optimised to follow schedule requirements of given targets at different observatories, but for this example, we will just create 20 random times between the ranges in which our time-series have been created.

In [ ]:
#Create random times for our i1 instance
t1 = np.random.uniform(min(i1.time),max(i1.time),20)
#Let us create the data at times t1
i1.create_data(t=t1)
#Let us plot where our data points are
i1.plot()

The previous plot shows the positions at which we have created our observations. We still need to add some white noise to make the situation more realistic. We do this by passing an error bar for each time-series that we have created. We will add an error bar for 0.001 for $S_1$ (RV), and 0.001 for $S_2$, as `err=[0.001,0.001]`.

In [ ]:
#The input vector err has to have one white noise term per each time-series
i1.add_white_noise(err=[0.003,0.003])
i1.plot()

Now our istance i1 has 20 synthetic observations for $S_1$ (RV) and $S_2$ with error bars of 0.001 for each datum. Now let's do something similar for our instrument i2.

### Data for our i2 instrument

Now it is time to create data taken at the times $\mathbf{t}$ for our instrument i2. The times vector $\mathbf{t}$ can be optimised to follow schedule requirements of given targets at different observatories, but for this example, we will just create 30 random times between the ranges in which our time-series have been created.

In [ ]:
#Create 30 random times for our i2 instance
t2 = np.random.uniform(min(i1.time),max(i1.time),30)
#Let us create the data at times t1
i2.create_data(t=t2)
#Let us plot where our data points are
i2.plot()

The previous plot shows the positions at which we have created our observations. We still need to add some white noise to make the situation more realistic. We do this by passing an error bar for each time-series that we have created. We will assume that our instrument i2 provide observations with higher error bars that s1, so we will add an error bar for 0.005 for $S_1$ (RV), and 0.005 for $S_2$, as `err=[0.005,0.005]`.

In [ ]:
#The input vector err has to have one white noise term per each time-series
i2.add_white_noise(err=[0.005,0.005])
i2.plot()

Now our istance i2 has 30 synthetic observations for $S_1$ (RV) and $S_2$ with error bars of 0.005 for each datum. If we look carefully, we will notice how some synthetic observations are clearly away from the model.

### Combining our data sets

At this point we have simulated data for our two fictional instruments i1 and i2. We want to combine the data sets to pass it to `pyaneti`. First let us see how our data looks combined. Let us plot the RV time-series to see how our data looks for both instruments.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.plot(i1.time,i1.rvs,'k',label='Model')
plt.plot(i1.time_data,i1.rvs_data,'o',label='i1')
plt.plot(i2.time_data,i2.rvs_data,'o',label='i2')
plt.xlabel('time [days]')
plt.ylabel('RV')
plt.legend()
plt.show()

We can do the same for our activity indicator time-series

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(i1.time,i1.s2,'k',label='Model')
plt.plot(i1.time_data,i1.s2_data,'o',label='i1')
plt.plot(i2.time_data,i2.s2_data,'o',label='i2')
plt.xlabel('time [days]')
plt.ylabel('S2')
plt.legend()
plt.show()

Let us now create the data file that we need to pass to pyaneti. The only requirement for pyaneti is that every time-series for every instrument have a different label. We can do this easily by naming rv_i1 and s2_i1 for the time-series of instrument i1, and rv_i2 and s2_i2 for the time-series of instrument i2.

In [ ]:
with open('2inst_data.dat','w') as f:
    #Save the rvs for i1
    for i in range(len(i1.time_data)):
        f.write('{:4.6f} {:4.6f} {:4.6f} rv_i1 \n'.format(i1.time_data[i],i1.rvs_data[i],i1.rvs_err))
    #Save the s2 for i1
    for i in range(len(i1.time_data)):
        f.write('{:4.6f} {:4.6f} {:4.6f} s2_i1 \n'.format(i1.time_data[i],i1.s2_data[i],i1.s2_err))
    #Save the rvs for i2
    for i in range(len(i2.time_data)):
        f.write('{:4.6f} {:4.6f} {:4.6f} rv_i2 \n'.format(i2.time_data[i],i2.rvs_data[i],i2.rvs_err))
    #Save the s2 for i2
    for i in range(len(i2.time_data)):
        f.write('{:4.6f} {:4.6f} {:4.6f} s2_i2 \n'.format(i2.time_data[i],i2.s2_data[i],i2.s2_err))

Now our file `'2inst_data.dat'` has the data formated as needed to be run with `pyaneti`